<a href="https://colab.research.google.com/github/nmalby/APS/blob/rewrite/APS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#APS Explorer

In [34]:
import math
from functools import lru_cache

##Parameters

In [ ]:
'''
Parameters
  Configuration
    Shell
      Loader      - Belt, 1, 2, 3, 4, 6, 7, 8             - Multiple            CheckBox
      Modules
        Head      - AP, Heavy, AP, Thump, Chem, Special   - Multiple            Checkbox
        Fuse      - None, Single, Double                  - Single              Radio
        Base      - None, BB, Trace, Cavitate             - Single              Radio
    Gun
      Gauge       - 18-500                                - Single              Field/Slider
      Barrels     - 1-6                                   - Single              Radio
      Evacuator   - Y/N                                   - Single              Radio
      Ejector     - Y/N                                   - Single              Radio
      Rail        - Y/N                                   - Single              Radio
      Rail Limit  - 1000-20000                            - Single              Field/Slider

    Engine
      PPM         - 200-800                               - Single              Field/Slider
      PPV         - 10-200                                - Single              Field/Slider
      PPB         - 10-200                                - Single              Field/Slider

  Requirements
    AP            - 0-60                                  - Single              Field/Slider
    Velocity      - 500-2000                              - Multiple (min-max)  Slider
    (Pen Req)     -                                       - Single              Field

  Goal Weights
    KD            - 0-1                                   - Single              Field/Slider
    CD            - 0-1                                   - Single              Field/Slider
    ROF           - 0-1                                   - Single              Field/Slider
    (Uptime)      - 0-1 (Belt)                            - Single              Field/Slider
    Volume        - 0-1                                   - Single              Field/Slider
    Cost          - 0-1 (Build + 10min)                   - Single              Field/Slider
'''

In [36]:
constraints = {
    "Configuration": {
        "Shell": {
            "Modules": {
                "Projectile": {
                    "Head": ["AP", "Heavy", "Sabot", "Thump", "Chem", "Special"],
                    "Fuse": ["None", "Single", "Double"],
                    "Base": ["None", "BB", "Trace", "Supercav"]
                }
            }
        },
        "Gun": {
            "Loader": ["Belt", 1, 2, 3, 4, 6, 7, 8],
            "Gauge": (18, 500),
            "Barrels": (1, 6),
            "Evacuator": ["Y", "N"],
            "Ejector": ["Y", "N"],
            "Rail": ["Y", "N"],
            "RailLimit": (1000, 20000),
            "ClipPer": (0, 4),
            "BeltInput": (1, 6)
        },
        "Engine": {
            "PPM": (200, 800),
            "PPV": (10, 200),
            "PPB": (10, 200)
        },
        "Duration": (0, 60)
    },
    "Requirements": {
        "AP": (0, 100),
        "Velocity": {
            "min": 500,
            "max": 2000
        }
    },
    "GoalWeights": {
        "KD": (0, 1),
        "CD": (0, 1),
        "RoF": (0, 1),
        "Uptime": (0, 1),
        "Volume": (0, 1),
        "Cost": (0, 1)
    }
}

In [37]:
parameters = {
    "Configuration": {
        "Shell": {
            "Modules": {
                "Projectile": {
                    "Head": "AP",
                    "Body": {
                        "SolidBody": 0,
                        "ChemBody": 0,
                        "Fuse": 0,
                    },
                    "Base": None
                },
                "Casing": {
                    "GP": 10,
                    "RC": 0
                }
            }
        },
        "Gun": {
            "Loader": [4],
            "Gauge": {
                "min": 100,
                "max": 250
            },
            "Barrels": 1,
            "Evacuator": False,
            "Ejector": False,
            "Rail": False,
            "RailLimit": 1000,
            "ClipPer": 2,
            "BeltInput": 2
        },
        "Engine": {
            "PPM": 500,
            "PPV": 100,
            "PPB": 100
        },
        "Duration": 10
    },
    "Requirements": {
        "AP": 20,
        "Velocity": {
            "min": 500,
            "max": 2000
        },
        "PenReq": None
    },
    "GoalWeights": {
        "KD": 0.5,
        "CD": 0.5,
        "RoF": 0.5,
        "Uptime": 0.5,
        "Volume": 0.5,
        "Cost": 0.5
    }
}

##Implementation

In [49]:
class Shell:
    def __init__(self, configuration):
        """
        Initialize the Shell object with the given configuration.

        :param configuration: Dictionary containing shell, gun, and engine configurations.
        """

        # Shell Configuration
        self.projectile = configuration["Configuration"]["Shell"]["Projectile"]
        self.casing = configuration["Configuration"]["Shell"]["Casing"]

        # Gun Configuration
        self.loader = configuration["Configuration"]["Gun"]["Loader"]
        self.gauge = configuration["Configuration"]["Gun"]["Gauge"]
        self.barrels = configuration["Configuration"]["Gun"]["Barrels"]
        self.evacuator = configuration["Configuration"]["Gun"]["Evacuator"]
        self.ejector = configuration["Configuration"]["Gun"]["Ejector"]
        self.rail = configuration["Configuration"]["Gun"]["Rail"]
        self.rail_limit = configuration["Configuration"]["Gun"]["RailLimit"]
        self.rail_draw_scalar = configuration["Configuration"]["Gun"]["RailDrawScalar"]
        self.clip_per = configuration["Configuration"]["Gun"]["ClipPer"]
        self.belt_input = configuration["Configuration"]["Gun"]["BeltInput"]

        # Engine Configuration
        self.engine = configuration["Configuration"]["Engine"] # PPM, PPV, PPB, PPC

        # Misc
        self.duration = configuration["Configuration"]["Duration"]

    lur_cache_size = None
    aps_mod = 23

    #   KD

    def kinetic_modifier(self):
        return self.weighted_modifier("KineticMod", 1.0)

    def ap_modifier(self):
        return self.weighted_modifier("ArmorPierceMod", 1.0)

    def velocity_modifier(self):
        return self.weighted_modifier("VelocityMod", 0.7) + (0.15 if self.projectile["Base"] == "BaseBleeder" else 0)

    def velocity(self):
        return math.sqrt(self.recoil() * 85 * self.gauge / self.gauge_coefficient() / self.projectile_length()) * self.velocity_modifier()

    def kinetic_damage(self):
        kd = 0
        if self.projectile["Head"] == "HP":
            kd = self.gauge_coefficient() * self.effective_projectile_module_count() * self.velocity() * self.kinetic_modifier() * 0.16 * self.aps_mod
        else:
            kd = (500 / max(self.gauge, 100))**0.15 * self.gauge_coefficient() * self.effective_projectile_module_count() * self.velocity() * self.kinetic_modifier() * 0.16 * self.aps_mod
        return kd

    def ap(self):
        return self.velocity() * self.ap_modifier() * 0.0175

    def effective_kinetic(self):
        return self.kinetic_damage() * self.ap()

    #   CD

    def chem_modifier(self):
        return self.weighted_modifier("ChemMod", 1.0)

    def chem_damage(self):
        chem_modules = self.projectile["Body"].get("ChemBody", 0)
        if self.projectile["Head"] == "Special":
            chem_modules += 0.2
        elif self.projectile["Head"] == "ChemHead":
            chem_modules += 1

        raw_chem = 3000 * (self.gauge_coefficient() * chem_modules * self.aps_mod * self.chem_modifier() / 25)**0.9
        chem_radius = min(raw_chem**0.3, 30)
        # Multiply by volume to approximate applied damage; divide by 1000 to make result more manageable
        chem_volume = chem_radius**3 * math.pi * 4 / 3
        return raw_chem * chem_volume / 1000

    def he_damage(self):
        chem_modules = self.projectile["Body"].get("ChemBody", 0)
        if self.projectile["Head"] == "Special":
            return 0
        elif self.projectile["Head"] == "ChemHead":
            chem_modules += 1

        raw_he = 3000 * (self.gauge_coefficient() * chem_modules * self.aps_mod * self.chem_modifier() / 25)**0.9
        he_radius = min(raw_he**0.3, 30)
        # Multiply by volume to approximate applied damage; divide by 1000 to make result more manageable
        he_volume = he_radius**3 * math.pi * 4 / 3
        return raw_he * he_volume / 1000

    def frag_damage(self):
        theta = 60  # Compromise of narrow and wide, avg angle multi
        angle_multi = (2 + math.sqrt(theta)) / 16

        chem_modules = self.projectile["Body"].get("ChemBody", 0)
        if self.projectile["Head"] == "ChemHead":
            chem_modules += 1

        raw_frag = self.gauge_coefficient() * chem_modules * self.chem_modifier() * 3000 * self.aps_mod
        frag_count = math.floor(raw_frag**0.25)

        return raw_frag * angle_multi / frag_count

    def special_damage(self):
        if self.projectile["Head"] == "Special":
            chem_modules = self.projectile["Body"].get("ChemBody", 0) + 0.8

            spec_coef = (1449 * (2 + math.sqrt(30))) / (16 * math.sqrt(0.5))

            return self.gauge_coefficient() * chem_modules * self.chem_modifier() * self.aps_mod * spec_coef
        else:
            return 0

    #   RoF

    @lru_cache(maxsize=lur_cache_size)
    def reload_time(self):
        reload = 17.5 * (self.gauge / 500)**1.35 * (2 + self.effective_shell_module_count())

        return reload if self.loader != "Belt" else reload * 1 # TODO: Calc

    def uptime(self):
        if self.loader != "Belt": return 1
        else:
            firing_cycle = 1 # TODO: Calc
            loading_cycle = 1 # TODO: Calc
            return firing_cycle/(loading_cycle + firing_cycle)

    def cooldown_time(self):
        return max(32.8215 * (self.gauge / 500)**1.35 * self.casing["GP"]**0.35, 0)

    #   Volume

    def loader_volume(self):
        ejector_volume = 2 if self.ejector else 0
        return (self.loader + 1 + ejector_volume) if self.loader != "Belt" else (2 + self.belt_input + ejector_volume)

    def recoil_volume(self):
        return self.recoil() / (self.reload_time() * 120)

    def cooler_volume(self):
        if self.casing["GP"] == 0: return 0
        multiBarrelPenalty = 1 + (self.barrels - 1) * 0.2
        boreEvacuatorBonus = 0

        if self.evacuator: boreEvacuatorBonus = (0.15 / (0.35355 / math.sqrt(self.gauge / 1000))) * multiBarrelPenalty / self.barrels

        return ((((self.cooldown_time() * multiBarrelPenalty) / self.reload_time() - boreEvacuatorBonus) /
            (1 + self.barrels * 0.05) / multiBarrelPenalty) * math.sqrt(self.gauge / 1000)) / 0.176775

    def charger_volume(self):
        return self.rail_draw() / self.reload_time() / 200

    def magnet_volume(self):
        return self.rail_draw() / 5000

    def engine_volume(self):
        return self.rail_draw() / self.reload_time() / self.engine["PPV"]

    def volume_per_intake(self):
        return max(
            self.loader_volume() +
            self.recoil_volume() +
            self.cooler_volume() +
            self.charger_volume() +
            self.magnet_volume() +
            self.engine_volume(),
            2
        )

    #   Cost

    def loader_cost(self):
        ejector_cost = cost_data["Ejector"] if self.ejector else 0
        if self.loader != "Belt": # compute for 1 intake, but fractionally distribute cost between autoloader and clip
            clip_fraction = self.clip_per / (1 + self.clip_per)
            return ((1 - clip_fraction) * (cost_data["Loader"][self.loader] + ejector_cost) +
                    clip_fraction * cost_data["Clip"][self.loader] + cost_data["InputFeeder"])
        else:
            return cost_data["Loader"]["Belt"] + cost_data["Clip"][1] + self.belt_input * cost_data["InputFeeder"]

    def build_cost(self):
        return (
            self.loader_cost() +
            self.recoil_volume() * cost_data["RecoilAbsorber"] +
            self.cooler_volume() * cost_data["Cooler"] +
            self.charger_volume() * cost_data["RailCharger"] +
            self.magnet_volume() * cost_data["RailMagnet"] +
            self.engine_volume() * self.engine["PPC"]
        )

    def projectile_cost(self):
        return ((17.413 * self.effective_shell_module_count() * (self.gauge/1000)**1.8)
                + self.rail_draw() / self.engine["PPM"])

    def costPerIntake(self):
        return self.build_cost() + (self.projectile_cost() / self.reload_time()) * self.duration * 60

    #   Misc

    def loaderLength(self):
        return 1000 if self.loader == "Belt" else (self.loader * 1000)

    def projectile_length(self):
        length = 0
        for kind, module in self.projectile.items():
            if module is not None:
                if isinstance(module, dict):  # Check if the module is a dictionary containing counts
                    for mod_name, count in module.items():
                        max_length = module_data.get(kind, {}).get(mod_name, {}).get("MaxLength", 500)
                        length += min(self.gauge, max_length) * count  # Multiply by the count
                else:
                    max_length = module_data.get(kind, {}).get(module, {}).get("MaxLength", 500)
                    length += min(self.gauge, max_length)
        return length

    def casingLength(self):
        return (self.casing["GP"] + (self.casing["RC"] or 0)) * self.gauge

    def totalLength(self): # will be used to apply penalty if totalLength > loaderLength
        return self.projectile_length() + self.casingLength()

    def effective_projectile_module_count(self):
        return self.projectile_length() / self.gauge

    def effective_shell_module_count(self):
        return self.effective_projectile_module_count() + (self.casing["GP"] + (self.casing["RC"] or 0)) / 4

    def weighted_modifier(self, modifier, ghostWeight):
        weighted_mod = 0.0

        head_module = self.projectile.get("Head")
        head_data = module_data.get("Head", {}).get(head_module, {})
        head_length = min(self.gauge, head_data.get("MaxLength", 500))

        body_length = self.projectile_length() - head_length
        effective_body_length = max(body_length, 2 * self.gauge)

        def update_weighted_mod(kind, mod_name, count=1):
            nonlocal weighted_mod
            mod_data = module_data.get(kind, {}).get(mod_name, {})
            mod_value = mod_data.get(modifier, 0)
            max_length = mod_data.get("MaxLength", 500)
            return count * mod_value * min(self.gauge, max_length)

        for kind, module in self.projectile.items():
            if kind != "Head" and module is not None:
                if isinstance(module, dict):
                    for mod_name, count in module.items():
                        weighted_mod += update_weighted_mod(kind, mod_name, count)
                else:
                    weighted_mod += update_weighted_mod(kind, module)

        # Add ghost modules and normalize, penalizes short shells
        weighted_mod += ghostWeight * (effective_body_length - body_length)

        weighted_mod /= effective_body_length

        weighted_mod *= head_data.get(modifier)

        return weighted_mod


    @lru_cache(maxsize=lur_cache_size)
    def rail_draw(self):
        return (min(self.rail_limit, self.max_draw()) * self.rail_draw_scalar) if self.rail else 0

    def max_draw(self):
        return 12500 * self.gauge_coefficient(0.65) * (self.effective_projectile_module_count() + (0.5 * self.casing["RC"]))

    def recoil(self):
        return self.gauge_coefficient(0.65) * self.casing["GP"] * 2500 + self.rail_draw()


    def gauge_coefficient(self, exponent=0.6):
        return (self.gauge**3/500**3)**exponent

###Test

In [50]:
cnfg = {
    "Configuration": {
        "Shell": {
            "Projectile": {
                "Head": "AP",
                "Body": {
                    "SolidBody": 0,
                    "ChemBody": 6,
                    "Fuse": 1,
                },
                "Base": "BaseBleeder",
            },
            "Casing": {
                "GP": 6,
                "RC": 0,
            }
        },
        "Gun": {
            "Loader": 8,
            "Gauge": 500,
            "Barrels": 1,
            "Evacuator": False,
            "Ejector": False,
            "Rail": True,
            "RailLimit": 67000,
            "RailDrawScalar": 1,
            "ClipPer": 2,
            "BeltInput": 2
        },
        "Engine": {
            "PPM": 500,
            "PPV": 100,
            "PPB": 100,
            "PPC": 0.1,
        },
        "Duration": 10
    },
}

test_shell = Shell(cnfg)
print(test_shell.gauge_coefficient())
print(test_shell.he_damage())

1.0
314293.0779842716


##Data

In [42]:
loaderLengths = ["Belt", 1, 2, 3, 4, 6, 8]

module_data = {
    "Head": {
        "AP": {
            "VelocityMod": 1.6,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.65,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "ShortName": "AP",
        },
        "Sabot": {
            "VelocityMod": 1.6,
            "KineticMod": 0.85,
            "ArmorPierceMod": 2.5,
            "ChemMod": 0.25,
            "AccMod": 1.0,
            "ShortName": "Sab",
        },
        "Heavy": {
            "VelocityMod": 1.45,
            "KineticMod": 1.75,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "ShortName": "Hvy",
        },
        "HP": {
            "VelocityMod": 1.45,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.2,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "ShortName": "HP",
        },
        "Chem": {   # HE, Frag, and EMP share stats
            "VelocityMod": 1.45,
            "KineticMod": 1.2,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "IsChem": True,
            "ShortName": "Chem",
        },
        "Flak": {
            "VelocityMod": 1.45,
            "KineticMod": 1.0,
            "ArmorPierceMod": 0.1,
            "ChemMod": 1.0,
            "IsChem": True,
            "ShortName": "Flak",
        },
        "Special": {   # HEAT and HESH share stats
            "VelocityMod": 1.6,
            "KineticMod": 0.1,
            "ArmorPierceMod": 0.1,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "IsChem": True,
            "ShortName": "Spc",
        }
    },
    "Body": {
        "SolidBody": {
            "VelocityMod": 1.1,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "ShortName": "Sol",
        },
        "SabotBody": {
            "VelocityMod": 1.1,
            "KineticMod": 0.8,
            "ArmorPierceMod": 1.4,
            "ChemMod": 0.25,
            "AccMod": 1.0,
            "ShortName": "Sab",
        },
        "ChemBody": {   # HE, Frag, and EMP share stats
            "VelocityMod": 1.0,
            "KineticMod": 1.0,
            "ArmorPierceMod": 0.8,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "IsChem": True,
            "ShortName": "Chem",
        },
        "FlakBody": {
            "VelocityMod": 1.0,
            "KineticMod": 1.0,
            "ArmorPierceMod": 0.1,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "IsChem": True,
            "ShortName": "Flak",
        },
        "FinBody": {
            "VelocityMod": 0.95,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "AccMod": 0.2,
            "MaxLength": 300,
            "ShortName": "Fin",
        },
        "Fuse": {   # Generic, will differentiate later
            "VelocityMod": 1.0,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "MaxLength": 100,
            "ShortName": "Fuse",
        },
        "UpperBound": {  # Highest values from Solid/Sabot/Chem
            "VelocityMod": 1.1,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.4,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "ShortName": "UpperBound",
        },
        "LowerBound": {  # Lowest values from Solid/Sabot/Chem
            "VelocityMod": 0.95,
            "KineticMod": 0.8,
            "ArmorPierceMod": 0.1,
            "ChemMod": 0.25,
            "AccMod": 0.2,
            "ShortName": "LowerBound",
        },
    },
    "Base": {
        "BaseBleeder": {
            "VelocityMod": 1.0,  # special +0.15 bonus handled elsewhere
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "MaxLength": 100,
            "ShortName": "BB",
        },
        "Tracer": {
            "VelocityMod": 1.0,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 1.0,
            "AccMod": 1.0,
            "MaxLength": 100,
            "ShortName": "Tracer",
        },
        "Supercav": {
            "VelocityMod": 1.0,
            "KineticMod": 1.0,
            "ArmorPierceMod": 1.0,
            "ChemMod": 0.75,
            "AccMod": 1.0,
            "MaxLength": 100,
            "ShortName": "Supercav",
        },
    }
}


In [43]:
cost_data = {
    "InputFeeder": 50,
    "Cooler": 50,
    "RecoilAbsorber": 80, # 80 per meter, longer ones scale linearly
    "RailCharger": 400,
    "RailMagnet": 300,
    "Ejector": 10,
    "Loader": {
        "Belt": 600,
        1: 240,
        2: 300,
        3: 330,
        4: 360,
        5: 390,
        6: 420,
        7: 450,
        8: 480,
    },
    "Clip": {
        "Belt": 160,
        1: 160,
        2: 200,
        3: 220,
        4: 240,
        5: 260,
        6: 280,
        7: 300,
        8: 320,
    }
}